In [1]:
####     //      Import/Install packages/REST API       \\       ####
# Default packages
import time
import csv
import os
import json

# Preinstalled packages
import requests
import pandas as pd

# Define desired wd, where I want to save my .csv files
desired_wd = '/Users/jenskoning/Documents/Python_projects/dsr_media_nlp/dsr_media_nlp/Script/'
os.chdir(desired_wd)

# URL of our News API
base_url = 'https://api.newscatcherapi.com/v2/search'

# My API key
X_API_KEY = 'uPGwd9h_DeqHKvxrDL5z9fu3t1Mc9rroBabtvX0iOuE'

In [3]:
####     //      Query Data       \\       ####
# Put my API key to headers in order to be authorized to perform a call
headers = {'x-api-key': X_API_KEY}

params = [
    {
    'q': 'Digital Silk Road AND DSR',
    'lang': 'en',
    'countries': 'MY,PH,ID,VN,TH,LA,KH,MM,SG,BN,TL',
    'to_rank': 10000,
    'page_size': 100,
    'sort_by': 'rank',
    'from': '3 years ago', 
    'page': 1
    },
    {
    'q': 'Digital Silk Road AND China',
    'lang': 'en',
    'countries': 'MY,PH,ID,VN,TH,LA,KH,MM,SG,BN,TL',
    'to_rank': 10000,
    'page_size': 100,
    'sort_by': 'rank',
    'from': '3 years ago', 
    'page': 1
    }
]


In [4]:
# Variable to store all found news articles, mp stands for "multiple queries"
all_news_articles_mp = []

# Infinite loop which ends when all articles are extracted
for separated_param in params:

    print(f'Query in use => {str(separated_param)}')
    
    while True:
        # Wait for 1 second between each call
        time.sleep(1)

        # GET Call from previous section enriched with some logs
        response = requests.get(base_url, headers=headers, params=separated_param)
        results = json.loads(response.text.encode())
        if response.status_code == 200:
            print(f'Done for page number => {separated_param["page"]}')


            # Adding your parameters to each result to be able to explore afterwards
            for i in results['articles']:
                i['used_params'] = str(separated_param)


            # Storing all found articles
            all_news_articles_mp.extend(results['articles'])

            # Ensuring to cover all pages by incrementing "page" value at each iteration
            separated_param['page'] += 1
            if separated_param['page'] > results['total_pages']:
                print("All articles have been extracted")
                break
            else:
                print(f'Proceed extracting page number => {separated_param["page"]}')
        else:
            print(results)
            print(f'ERROR: API call failed for page number => {separated_param["page"]}')
            break

print(f'Number of extracted articles => {str(len(all_news_articles_mp))}')

Query in use => {'q': 'Digital Silk Road AND DSR', 'lang': 'en', 'countries': 'MY,PH,ID,VN,TH,LA,KH,MM,SG,BN,TL', 'to_rank': 10000, 'page_size': 100, 'sort_by': 'rank', 'from': '3 years ago', 'page': 1}
Done for page number => 1
All articles have been extracted
Query in use => {'q': 'Digital Silk Road AND China', 'lang': 'en', 'countries': 'MY,PH,ID,VN,TH,LA,KH,MM,SG,BN,TL', 'to_rank': 10000, 'page_size': 100, 'sort_by': 'rank', 'from': '3 years ago', 'page': 1}
Done for page number => 1
Proceed extracting page number => 2
Done for page number => 2
Proceed extracting page number => 3
Done for page number => 3
All articles have been extracted
Number of extracted articles => 222


In [5]:
# Define variables
unique_ids = []
all_news_articles = []

# Iterate on each article and check whether we saw this _id before
for article in all_news_articles_mp:
    if article['_id'] not in unique_ids:
        unique_ids.append(article['_id'])
        all_news_articles.append(article)


In [6]:
# Generate CSV from Pandas table
# Create Pandas table
pandas_table = pd.DataFrame(all_news_articles)

# Generate CSV
pandas_table.to_csv('extracted_news_articles_allsea.csv', encoding='utf-8', sep=';')

In [31]:
pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 98 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 18.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import Levenshtein
# Function to remove approximate duplicates from a DataFrame
def remove_approx_duplicates(df, column_name, threshold=3):
    cleaned_df = pd.DataFrame(columns=df.columns)
    for _, row in df.iterrows():
        is_duplicate = False
        for index, cleaned_row in cleaned_df.iterrows():
            distance = Levenshtein.distance(row[column_name], cleaned_row[column_name])
            if distance <= threshold:
                is_duplicate = True
                break
        if not is_duplicate:
            cleaned_df = cleaned_df.append(row)
    return cleaned_df

# Remove approximate duplicates based on the "title" column
cleaned_df = remove_approx_duplicates(df, column_name="title")

# Print the cleaned DataFrame
print(cleaned_df)